In [2]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI

from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnableLambda
import Helper
load_dotenv()

google_api_jey = os.getenv("GOOGLE_API_KEY")
oprn_api_key = os.getenv("OPEN_API_KEY")

google_llm = ChatGoogleGenerativeAI(temperature=0, api_key=google_api_jey, model="gemini-2.5-pro")

open_llm = ChatOpenAI(
    model="gpt-3.5-turbo", 
    temperature=0, 
    api_key=oprn_api_key,
    )


Manual trimming history

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.memory import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

prompt_template = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer all the questions very shortly"),
    MessagesPlaceholder(variable_name="history"),
    ("human","{input}"),
])

chain = prompt_template |  open_llm | StrOutputParser()

conversation ={}

def chat_with_bot(payload):
    input = payload['input']
    session_id = payload['session_id']

    if session_id not in conversation:
        conversation[session_id] = ChatMessageHistory() 
    
    history = conversation[session_id]

    history.messages = history.messages[-2:]    

    history.add_user_message(input)

    res = chain.invoke({"input": input, "history": history.messages}) 
    history.add_ai_message(res)

    print("Conversation history:", history.messages)

    return res

res = chat_with_bot({"input" : "My name is Sampath", "session_id" : "user1"})
print(conversation, "\n", res, "\n")

res = chat_with_bot({"input" : "What is Google", "session_id" : "user1"})
print(conversation, "\n", res, "\n")

res = chat_with_bot({"input" : "Where is it located", "session_id" : "user1"})
print(conversation, "\n", res, "\n")

res = chat_with_bot({"input" : "What is my name", "session_id" : "user1"})
print(conversation, "\n", res, "\n")

Conversation history: [HumanMessage(content='My name is Sampath', additional_kwargs={}, response_metadata={}), AIMessage(content='Nice to meet you, Sampath! How can I assist you today?', additional_kwargs={}, response_metadata={})]
{'user1': InMemoryChatMessageHistory(messages=[HumanMessage(content='My name is Sampath', additional_kwargs={}, response_metadata={}), AIMessage(content='Nice to meet you, Sampath! How can I assist you today?', additional_kwargs={}, response_metadata={})])} 
 Nice to meet you, Sampath! How can I assist you today? 

Conversation history: [HumanMessage(content='My name is Sampath', additional_kwargs={}, response_metadata={}), AIMessage(content='Nice to meet you, Sampath! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is Google', additional_kwargs={}, response_metadata={}), AIMessage(content='Google is a multinational technology company that specializes in internet-related services and products, including s

Trimming with auto using dequeue